# リポジトリクローン

In [ ]:
!git clone https://github.com/GlobalFlowNet/GlobalFlowNet

In [ ]:
%cd GlobalFlowNet/Code

# 必要パッケージインストール

In [ ]:
!pip install sk-video

# サンプル動画ダウンロード

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1K9WSa4eOgCEwCVsWXKgdvHiN_kb4g6Qo', 'sample.mp4', quiet=False)

# 動画のフレーム長を取得

In [5]:
input_path = 'sample.mp4'
output_path = 'output.mp4'

In [ ]:
import cv2
video_capture = cv2.VideoCapture(input_path)
total_frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
video_capture.release()
print(total_frame_count)

# OptNet準備

In [7]:
import json

import numpy as np
from Utils.VideoUtility import VideoReader, VideoWriter
from GlobalFlowNets.GlobalPWCNets import getGlobalPWCModel

def getConfig(filePath):
    with open(filePath, 'r') as openfile:
        config = json.load(openfile)
    return config

inpVideo = VideoReader(input_path, maxFrames=total_frame_count)
outVideo = VideoWriter(output_path, fps=inpVideo.getFPS())

Stabilizers = {}
Stabilizers['GNetAffine'] = ['GNetAffine']
Stabilizers['MSPhotometric'] = ['MSPhotometric']
Stabilizers['GNetMSPhotometric'] = ['GNetAffine', 'MSPhotometric']

inpTag = 'Original'
outTag = 'GNetMSPhotometric'
modelTag = 'GLNoWarp4YTBB'

config = getConfig('GlobalFlowNets/trainedModels/config.json')['GlobalNetModelParameters']
OptNet = getGlobalPWCModel(config, 'GlobalFlowNets/trainedModels/GFlowNet.pth', loadInCPU=True)
OptNet = OptNet.eval().cuda()

# ComposedStabilizer準備

In [8]:
import torch
import numpy as np
from Stabilizers.MSPhotometric import MSPhotometric
from Stabilizers.JoinedAdaptiveGNetStabilizer import JoinedAdaptiveGNetStabilizer

class ComposedStabilizer():
    def __init__(self, Video, OptNet, stabilizers=[], span=11, cutoffFreq=5, crop=.8):
        self.Video = Video
        self.span = span
        self.frames = torch.from_numpy(np.transpose(Video.getFrames(), (0, 3, 1, 2))) / 255.0
        self.shape = [self.frames.shape[-2], self.frames.shape[-1]]
        self.optimalSpans = None
        self.cropFeild = None

        self.stabilizers = []
        for stabilizer in stabilizers:
            if stabilizer == 'GNetAffine':
                AffineStabilizer = JoinedAdaptiveGNetStabilizer(self.frames, OptNet, dSpan=32, crop=crop)
                self.stabilizers.append(AffineStabilizer)
                self.frames, self.cropFeild = AffineStabilizer.getStabilizedFrames()
            elif stabilizer == 'MSPhotometric':
                Stabilizer = MSPhotometric(self.frames, OptNet, span=13, cutoffFreq=5)
                Stabilizer.stabilizeFrames()

    def cvtTensor2Uint8(self, img):
        img = np.round(255 * np.transpose(img.numpy(), (1, 2, 0))).astype('uint8')
        return img

    def getStabilizedFrame(self, idx):
        return self.cvtTensor2Uint8(self.frames[idx])

# ブレ補正、動画出力

In [ ]:
maxAffineCrop = 0.8
stabilizer = ComposedStabilizer(inpVideo, OptNet, stabilizers=Stabilizers[outTag], crop=maxAffineCrop)

for f in range(inpVideo.getNrFrames()):
    outFrame = stabilizer.getStabilizedFrame(f)
    outVideo.writeFrame(outFrame)
outVideo.close()